## Лабораторна робота 3
## Спостерігається послідовність незалежних нормально розподілених випадкових величин x1,...,xn (наприклад, n=100) з дисперсією 1. Математичне сподівання кожної величини залежить від поточного стану системи. Припустимо, що для стану k=1, математичне сподівання рівне або a1=0 або a1=1, а для стану k=2, математичне сподівання рівне або a2=2 або a2=3. Нехай справжні математичні сподівання рівні a1=0, a2=3, стан k=1 виникає з ймовірністю 1/3, а стан k=2 – з ймовірністю 2/3. Потрібно згенерувати n незалежних випадкових величин з нормальним розподілом, причому з ймовірністю 1/3 їх математичне сподівання рівне 0, дисперсія рівна 1, а з ймовірністю 2/3 їх математичне сподівання рівне 3, дисперсія рівна 1. Починаючи з оцінок P(k=1)=P(k=2)=0.5, a1=1, a2=2 реалізувати алгоритм самонавчання. Умовою зупинки алгоритму можна вважати, що оцінки a1, a2 не змінюються, а оцінки P(k=1), P(k=2) відрізняються менше ніж на 0.001.

In [1]:
import numpy as np
from scipy.stats import norm

# Given: a sequence of independent normally distributed random variables x1,...,xn
# with variance 1 and mean depending on the current state of the system.
# For state k=1, mean a1=0, for state k=2, mean a2=3.
# State k=1 occurs with probability 1/3, and state k=2 with probability 2/3.
# Starting estimates: P(k=1)=P(k=2)=0.5, a1=1, a2=2.

def e_step(data, p1, p2, a1, a2, variance):
    """E-step of the EM algorithm, calculates the expected responsibilities."""
    likelihood_1 = p1 * norm.pdf(data, a1, np.sqrt(variance))
    likelihood_2 = p2 * norm.pdf(data, a2, np.sqrt(variance))
    total_likelihood = likelihood_1 + likelihood_2

    # responsibilities
    gamma_1 = likelihood_1 / total_likelihood
    gamma_2 = likelihood_2 / total_likelihood

    return gamma_1, gamma_2

def m_step(data, gamma_1, gamma_2):
    """M-step of the EM algorithm, updates the parameter estimates."""
    # update the probabilities
    p1 = np.mean(gamma_1)
    p2 = np.mean(gamma_2)

    # update the means
    a1 = np.sum(gamma_1 * data) / np.sum(gamma_1)
    a2 = np.sum(gamma_2 * data) / np.sum(gamma_2)

    return p1, p2, a1, a2

def em_algorithm(n=100, tol=0.001, max_iter=1000):
    """The EM algorithm."""
    # True parameters (unknown to the algorithm)
    true_p1, true_p2 = 1/3, 2/3
    true_a1, true_a2 = 0, 3
    variance = 1

    # Initial estimates
    p1, p2 = 0.5, 0.5
    a1, a2 = 1, 2

    # Generate data
    data = np.concatenate((np.random.normal(true_a1, np.sqrt(variance), int(n * true_p1)),
                           np.random.normal(true_a2, np.sqrt(variance), int(n * true_p2))))
    np.random.shuffle(data)  # Shuffle the data to ensure randomness

    for iteration in range(max_iter):
        # Store the old values for checking convergence
        old_p1, old_p2, old_a1, old_a2 = p1, p2, a1, a2

        # E-step: calculate responsibilities
        gamma_1, gamma_2 = e_step(data, p1, p2, a1, a2, variance)

        # M-step: update the parameter estimates
        p1, p2, a1, a2 = m_step(data, gamma_1, gamma_2)

        # Check convergence
        if max(abs(p1 - old_p1), abs(p2 - old_p2), abs(a1 - old_a1), abs(a2 - old_a2)) < tol:
            break

    return {'p1': p1, 'p2': p2, 'a1': a1, 'a2': a2, 'iterations': iteration+1}

# Run the EM algorithm
em_results = em_algorithm()
em_results


(0.3454763588142003,
 0.6545236411857998,
 0.09964443115484939,
 3.123132576450254,
 9)

Функція e_step:
        Для кожного спостереження у даних розраховується ймовірність (likelihood) того, що воно належить до першого розподілу (з параметрами a1a1 та варіансом 1) та до другого розподілу (з параметрами та варіансом 1).
        Ці ймовірності нормалізуються для того, щоб отримати "відповідальності" та, які представляють ймовірність того, що кожне спостереження належить до першого чи другого розподілу відповідно.

Функція m_step:
        Використовуючи відповідальності з попереднього кроку, оновлюються оцінки ймовірностей станів P(k=1)P(k=1) та P(k=2)P(k=2) як середнє значення відповідальностей.
        Оновлюються оцінки математичних сподівань для кожного з розподілів як зважені середні значення спостережень, де ваги це відповідальності.

Функція em_algorithm:
        Ініціалізуються початкові параметри для ймовірностей та математичних сподівань.
        Генерується набір даних з нормальних розподілів з істинними значеннями параметрів, які алгоритм намагається оцінити.
        Виконується цикл, який чергує E-крок та M-крок до тих пір, поки зміни в параметрах не стануть меншими за заданий поріг toltol або не буде досягнуто максимальну кількість ітерацій.
        У кожній ітерації зберігаються старі значення параметрів, щоб можна було порівняти їх із новими оцінками та перевірити, чи алгоритм досягнув збіжності.